<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Metricas/MetricasLast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the ultralytics package using pip
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 637.4 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [2]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image

In [4]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile

# Función para crear carpetas si no existen
def create_directories_if_not_exist(*directories):
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

# Ruta al archivo ZIP y la ruta de destino
zip_file_path = '/content/drive/MyDrive/TFM/models/YoloV8_250Epoch_V1.zip'
output_directory = '/content/models/'

# Crear el directorio de salida si no existe
create_directories_if_not_exist(output_directory)

# Descomprimir el archivo ZIP en el directorio de salida
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_directory)

print(f"Archivo descomprimido en: {output_directory}")


In [ ]:
import shutil
import os

# Función para borrar directorios, incluso si tienen contenido
def delete_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
        print(f"Directorio '{directory}' ha sido eliminado.")
    else:
        print(f"El directorio '{directory}' no existe.")

# Especifica la ruta del directorio que quieres borrar
directory_to_delete = '/content/models/YoloV8_250Epoch_V1/trial_03'

# Llamada a la función para borrar el directorio
delete_directory(directory_to_delete)


In [ ]:
import pandas as pd

# Ruta al archivo CSV
csv_file_path = '/content/models/YoloV8_250Epoch_V1/trial_1/results.csv'

# Leer el archivo CSV
df = pd.read_csv(csv_file_path)

# Imprimir los nombres de las columnas
print("Nombres de las columnas en el archivo CSV:")
print(df.columns.tolist())


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob

def clean_column_names(df):
    # Eliminar espacios en blanco al inicio y al final de los nombres de las columnas
    df.columns = df.columns.str.strip()
    return df

def plot_metrics_from_trials(base_directory):
    # Listar todas las carpetas de trials
    trial_folders = glob.glob(os.path.join(base_directory, 'trial_*'))

    # Inicializar listas para almacenar los datos de todas las métricas
    precision_data = []
    recall_data = []
    map50_data = []

    # Recorrer cada carpeta de trial
    for folder in trial_folders:
        results_file = os.path.join(folder, 'results.csv')
        if os.path.exists(results_file):
            # Leer el archivo CSV
            df = pd.read_csv(results_file)

            # Limpiar nombres de columnas
            df = clean_column_names(df)

            # Imprimir los nombres de las columnas para depuración
            print(f"Columnas en '{results_file}': {df.columns.tolist()}")

            # Intentar guardar las columnas de interés
            try:
                precision_data.append(df['metrics/precision(M)'].values)
                recall_data.append(df['metrics/recall(M)'].values)
                map50_data.append(df['metrics/mAP50(M)'].values)
            except KeyError as e:
                print(f"Columna no encontrada: {e}")

    # Convertir listas a DataFrames para facilitar el cálculo de la media
    precision_df = pd.DataFrame(precision_data).transpose()
    recall_df = pd.DataFrame(recall_data).transpose()
    map50_df = pd.DataFrame(map50_data).transpose()

    # Calcular las medias
    precision_mean = precision_df.mean(axis=1)
    recall_mean = recall_df.mean(axis=1)
    map50_mean = map50_df.mean(axis=1)

    # Plotear las métricas
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    # Plot Precision
    for trial_data in precision_data:
        axs[0].plot(trial_data, color='lightblue', alpha=0.5)
    axs[0].plot(precision_mean, color='darkblue', label='Mean Precision', linewidth=2)
    axs[0].set_title('Precision Over Epochs')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Precision')
    axs[0].legend()

    # Plot Recall
    for trial_data in recall_data:
        axs[1].plot(trial_data, color='lightgreen', alpha=0.5)
    axs[1].plot(recall_mean, color='darkgreen', label='Mean Recall', linewidth=2)
    axs[1].set_title('Recall Over Epochs')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Recall')
    axs[1].legend()

    # Plot mAP50
    for trial_data in map50_data:
        axs[2].plot(trial_data, color='lightcoral', alpha=0.5)
    axs[2].plot(map50_mean, color='darkred', label='Mean mAP50', linewidth=2)
    axs[2].set_title('mAP50 Over Epochs')
    axs[2].set_xlabel('Epochs')
    axs[2].set_ylabel('mAP50')
    axs[2].legend()

    # Ajustar el espaciado
    plt.tight_layout()
    plt.show()

# Uso de la función
base_directory = '/content/models/YoloV8_250Epoch_V1'
plot_metrics_from_trials(base_directory)


In [ ]:
import pandas as pd
import glob
import os

def calculate_weighted_mean(last_epoch_metrics):
    """
    Calcula la media ponderada de las métricas de precisión, recall y mAP50.
    Los pesos para cada métrica son 1 en este caso, pero puedes ajustarlos si es necesario.
    """
    # Pesos de las métricas (se asume un peso igual para cada métrica en este caso)
    weights = {'metrics/precision(M)': 1, 'metrics/recall(M)': 1, 'metrics/mAP50(M)': 1}

    weighted_sum = 0
    total_weight = 0

    for metric, weight in weights.items():
        if metric in last_epoch_metrics:
            weighted_sum += last_epoch_metrics[metric] * weight
            total_weight += weight

    return weighted_sum / total_weight if total_weight > 0 else None

def analyze_trials(base_directory):
    trial_metrics = {}

    # Listar todas las carpetas de trials
    trial_folders = glob.glob(os.path.join(base_directory, 'trial_*'))

    # Recorrer cada carpeta de trial
    for folder in trial_folders:
        results_file = os.path.join(folder, 'results.csv')
        if os.path.exists(results_file):
            # Leer el archivo CSV
            df = pd.read_csv(results_file)

            # Limpiar nombres de columnas
            df = clean_column_names(df)

            # Obtener el último epoch
            last_epoch = df.iloc[-1]

            # Extraer las métricas del último epoch
            last_epoch_metrics = {
                'metrics/precision(M)': last_epoch.get('metrics/precision(M)', None),
                'metrics/recall(M)': last_epoch.get('metrics/recall(M)', None),
                'metrics/mAP50(M)': last_epoch.get('metrics/mAP50(M)', None)
            }

            # Calcular la media ponderada
            weighted_mean = calculate_weighted_mean(last_epoch_metrics)

            # Guardar en el diccionario con el número del trial como clave
            trial_number = int(os.path.basename(folder).split('_')[1])
            trial_metrics[trial_number] = weighted_mean

    # Ordenar los trials por la media ponderada (de mayor a menor) y obtener los 3 mejores
    sorted_trials = sorted(trial_metrics.items(), key=lambda x: x[1], reverse=True)
    best_3_trials = [trial[0] for trial in sorted_trials[:3]]

    # Imprimir resultados
    print("Media ponderada de las métricas para cada trial:")
    for trial_number, mean in trial_metrics.items():
        print(f"Trial {trial_number}: {mean}")

    print("\nLos 3 mejores trials en orden:")
    print(best_3_trials)

    return trial_metrics, best_3_trials

# Usar la función
base_directory = '/content/models/YoloV8_250Epoch_V1'
trial_metrics, best_3_trials = analyze_trials(base_directory)


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob

def clean_column_names(df):
    # Eliminar espacios en blanco al inicio y al final de los nombres de las columnas
    df.columns = df.columns.str.strip()
    return df

def plot_metrics_from_trials(base_directory, selected_trials):
    # Listar todas las carpetas de trials
    trial_folders = glob.glob(os.path.join(base_directory, 'trial_*'))

    # Inicializar listas para almacenar los datos de todas las métricas
    precision_data = []
    recall_data = []
    map50_data = []

    # Recorrer cada carpeta de trial
    for i, folder in enumerate(trial_folders):
        if (i + 1) in selected_trials:  # Filtrar solo los trials seleccionados
            results_file = os.path.join(folder, 'results.csv')
            if os.path.exists(results_file):
                # Leer el archivo CSV
                df = pd.read_csv(results_file)

                # Limpiar nombres de columnas
                df = clean_column_names(df)

                # Imprimir los nombres de las columnas para depuración
                print(f"Columnas en '{results_file}': {df.columns.tolist()}")

                # Intentar guardar las columnas de interés
                try:
                    precision_data.append(df['metrics/precision(M)'].values)
                    recall_data.append(df['metrics/recall(M)'].values)
                    map50_data.append(df['metrics/mAP50(M)'].values)
                except KeyError as e:
                    print(f"Columna no encontrada: {e}")

    # Convertir listas a DataFrames para facilitar el cálculo de la media
    precision_df = pd.DataFrame(precision_data).transpose()
    recall_df = pd.DataFrame(recall_data).transpose()
    map50_df = pd.DataFrame(map50_data).transpose()

    # Calcular las medias
    precision_mean = precision_df.mean(axis=1)
    recall_mean = recall_df.mean(axis=1)
    map50_mean = map50_df.mean(axis=1)

    # Plotear las métricas
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    # Plot Precision
    for trial_data in precision_data:
        axs[0].plot(trial_data, color='lightblue', alpha=0.5)
    axs[0].plot(precision_mean, color='darkblue', label='Mean Precision', linewidth=2)
    axs[0].set_title('Precision Over Epochs')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Precision')
    axs[0].legend()

    # Plot Recall
    for trial_data in recall_data:
        axs[1].plot(trial_data, color='lightgreen', alpha=0.5)
    axs[1].plot(recall_mean, color='darkgreen', label='Mean Recall', linewidth=2)
    axs[1].set_title('Recall Over Epochs')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Recall')
    axs[1].legend()

    # Plot mAP50
    for trial_data in map50_data:
        axs[2].plot(trial_data, color='lightcoral', alpha=0.5)
    axs[2].plot(map50_mean, color='darkred', label='Mean mAP50', linewidth=2)
    axs[2].set_title('mAP50 Over Epochs')
    axs[2].set_xlabel('Epochs')
    axs[2].set_ylabel('mAP50')
    axs[2].legend()

    # Ajustar el espaciado
    plt.tight_layout()
    plt.show()

# Uso de la función
base_directory = '/content/models/YoloV8_250Epoch_V1'
selected_trials = [5, 15, 1]
plot_metrics_from_trials(base_directory, selected_trials)


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob

def clean_column_names(df):
    # Eliminar espacios en blanco al inicio y al final de los nombres de las columnas
    df.columns = df.columns.str.strip()
    return df

def plot_best_trials_metrics(base_directory, best_trials):
    # Inicializar listas para almacenar los datos de las métricas
    precision_data = []
    recall_data = []
    map50_data = []

    # Recorrer los mejores trials
    for trial_number in best_trials:
        trial_folder = os.path.join(base_directory, f'trial_{trial_number}')
        results_file = os.path.join(trial_folder, 'results.csv')
        if os.path.exists(results_file):
            # Leer el archivo CSV
            df = pd.read_csv(results_file)

            # Limpiar nombres de columnas
            df = clean_column_names(df)

            # Guardar las columnas de interés
            precision_data.append(df['metrics/precision(M)'].values)
            recall_data.append(df['metrics/recall(M)'].values)
            map50_data.append(df['metrics/mAP50(M)'].values)

    # Convertir listas a DataFrames para facilitar el cálculo de la media
    precision_df = pd.DataFrame(precision_data).transpose()
    recall_df = pd.DataFrame(recall_data).transpose()
    map50_df = pd.DataFrame(map50_data).transpose()

    # Calcular las medias
    precision_mean = precision_df.mean(axis=1)
    recall_mean = recall_df.mean(axis=1)
    map50_mean = map50_df.mean(axis=1)

    # Plotear las métricas
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    # Plot Precision
    for trial_data in precision_data:
        axs[0].plot(trial_data, color='lightblue', alpha=0.5)
    axs[0].plot(precision_mean, color='darkblue', label='Mean Precision', linewidth=2)
    axs[0].set_title('Precision Over Epochs (Best Trials)')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Precision')
    axs[0].legend()

    # Plot Recall
    for trial_data in recall_data:
        axs[1].plot(trial_data, color='lightgreen', alpha=0.5)
    axs[1].plot(recall_mean, color='darkgreen', label='Mean Recall', linewidth=2)
    axs[1].set_title('Recall Over Epochs (Best Trials)')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Recall')
    axs[1].legend()

    # Plot mAP50
    for trial_data in map50_data:
        axs[2].plot(trial_data, color='lightcoral', alpha=0.5)
    axs[2].plot(map50_mean, color='darkred', label='Mean mAP50', linewidth=2)
    axs[2].set_title('mAP50 Over Epochs (Best Trials)')
    axs[2].set_xlabel('Epochs')
    axs[2].set_ylabel('mAP50')
    axs[2].legend()

    # Ajustar el espaciado
    plt.tight_layout()
    plt.show()

# Usar la función con los 3 mejores trials
base_directory = '/content/models/YoloV8_250Epoch_V1'
# Supongamos que has calculado los mejores 3 trials previamente
best_3_trials = [5, 15, 1]  # Reemplaza con los números reales de los mejores trials

plot_best_trials_metrics(base_directory, best_3_trials)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob

def clean_column_names(df):
    # Eliminar espacios en blanco al inicio y al final de los nombres de las columnas
    df.columns = df.columns.str.strip()
    return df

def plot_last_epoch_values(base_directory):
    # Listar todas las carpetas de trials
    trial_folders = sorted(glob.glob(os.path.join(base_directory, 'trial_*')), key=lambda x: int(os.path.basename(x).split('_')[1]))

    # Inicializar listas para almacenar los datos del último epoch
    all_precision_last_epoch = []
    all_recall_last_epoch = []
    all_map50_last_epoch = []

    # Recorrer cada carpeta de trial
    for folder in trial_folders:
        trial_number = int(os.path.basename(folder).split('_')[1])
        results_file = os.path.join(folder, 'results.csv')
        if os.path.exists(results_file):
            # Leer el archivo CSV
            df = pd.read_csv(results_file)

            # Limpiar nombres de columnas
            df = clean_column_names(df)

            # Obtener el último epoch
            last_epoch = df.iloc[-1]

            # Extraer las métricas del último epoch
            precision = last_epoch.get('metrics/precision(M)', None)
            recall = last_epoch.get('metrics/recall(M)', None)
            map50 = last_epoch.get('metrics/mAP50(M)', None)

            # Agregar datos a las listas
            all_precision_last_epoch.append(precision)
            all_recall_last_epoch.append(recall)
            all_map50_last_epoch.append(map50)

    # Graficar las métricas del último epoch
    fig, axs = plt.subplots(3, 1, figsize=(12, 18))

    # Plot Precision
    axs[0].plot(range(1, len(all_precision_last_epoch) + 1), all_precision_last_epoch, marker='o', linestyle='-', color='blue')
    axs[0].set_title('Precision at Last Epoch')
    axs[0].set_xlabel('Trial Number')
    axs[0].set_ylabel('Precision')
    axs[0].grid(True)

    # Plot Recall
    axs[1].plot(range(1, len(all_recall_last_epoch) + 1), all_recall_last_epoch, marker='o', linestyle='-', color='green')
    axs[1].set_title('Recall at Last Epoch')
    axs[1].set_xlabel('Trial Number')
    axs[1].set_ylabel('Recall')
    axs[1].grid(True)

    # Plot mAP50
    axs[2].plot(range(1, len(all_map50_last_epoch) + 1), all_map50_last_epoch, marker='o', linestyle='-', color='red')
    axs[2].set_title('mAP50 at Last Epoch')
    axs[2].set_xlabel('Trial Number')
    axs[2].set_ylabel('mAP50')
    axs[2].grid(True)

    # Ajustar el espaciado
    plt.tight_layout()
    plt.show()

# Ejecutar la función para graficar los valores del último epoch
base_directory = '/content/models/YoloV8_250Epoch_V1'
plot_last_epoch_values(base_directory)
